Arturo Serdan
CSC 405
10/05/2021
Assignment 2

This assignment is meant to merge three data files, being Users, Movies, and Ratings; in order to find the which are the the highest rated movies, as well as what are the 'best' movies. 
The first part, which involves the merging, it is crucial since the merged data file will be used for the other two tasks; I used an outer join because we have a many to many relationship between the movies, users and ratings.
In order to find the highest rated movie(s) I filter the movies by getting the average ratings and keeping the highest.
For the final part I decided to use 2 additional filters, age and number of reviews, along with the first filter used in the first task in order to obtain the 'best' movies given a specific critereia. 

# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [198]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [199]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding = "unicode_escape")
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames, encoding = "unicode_escape")
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames, encoding = "unicode_escape")

    return users, ratings, movies

In [200]:
users, ratings, movies = get_movie_data()

C:\Users\artur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [201]:
print(users.head())

   user_id gender  age  occupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455


In [202]:
print(ratings.head()) 

   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291


In [203]:
print(movies.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [204]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

0    Toy Story
1         1995
dtype: object

In [205]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [206]:
print(movies, ratings)

      movie_id                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
3878      3948             Meet the Parents (2000)   
3879      3949          Requiem for a Dream (2000)   
3880      3950                    Tigerland (2000)   
3881      3951             Two Family House (2000)   
3882      3952               Contender, The (2000)   

                            genres  year                  short_title  
0      Animation|Children's|Comedy  1995                    Toy Story  
1     Adventure|Children's|Fantasy  1995                      Jumanji  
2                   Comedy|Romance  1995             Grumpier Old Men  
3                     Comedy|Drama  1995            Waiting to 

##### Join the tables with `pd.merge` (20 pts)

In [207]:
df_m_user_rates = pd.merge(users, ratings, how='outer')
df_m_user_rates2 = df_m_user_rates.set_index(['movie_id'])
print(df_m_user_rates2.head())

          user_id gender  age  occupation    zip  rating  timestamp
movie_id                                                           
1193            1      F    1          10  48067       5  978300760
661             1      F    1          10  48067       3  978302109
914             1      F    1          10  48067       3  978301968
3408            1      F    1          10  48067       4  978300275
2355            1      F    1          10  48067       5  978824291


In [208]:
# Final merge of Movies, Users, and Ratings
df_merged = pd.merge(movies, df_m_user_rates2, how = 'outer', on= 'movie_id')

In [209]:
df_merged.head()

,movie_id,title,genres,year,short_title,user_id,gender,age,occupation,zip,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,1.0,F,1.0,10.0,48067,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,6.0,F,50.0,9.0,55117,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,8.0,M,25.0,12.0,11413,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,9.0,M,25.0,17.0,61614,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,10.0,F,35.0,1.0,95370,5.0,978226474.0


##### What's the highest rated movie? (20 pts))

In [210]:
movie_index = df_merged['short_title'].unique()
rate_averages_by_movie = df_merged.groupby('short_title').mean().reset_index()
print(rate_averages_by_movie.head())


              short_title  movie_id      user_id        age  occupation  \
0         $1,000,000 Duck    2031.0  2865.432432  31.162162    7.378378   
1           'Night Mother    3112.0  3231.714286  33.271429    6.242857   
2      'Til There Was You     779.0  2729.769231  28.826923    7.903846   
3             'burbs, The    2072.0  2954.333333  27.897690    8.148515   
4  ...And Justice for All    3420.0  3173.246231  35.984925    8.010050   

     rating     timestamp  
0  3.027027  9.782423e+08  
1  3.371429  9.715184e+08  
2  2.692308  9.728025e+08  
3  2.910891  9.718751e+08  
4  3.713568  9.707617e+08  


In [211]:
lowest_rated = 0
highest_rated = []
for i in range(0, 3840):
    if lowest_rated <=  rate_averages_by_movie.loc[i].rating:
        movie_rating_temp = [rate_averages_by_movie.loc[i].short_title, rate_averages_by_movie.loc[i].rating]
        if lowest_rated == 5:
            movie_rating_temp = [rate_averages_by_movie.loc[i].short_title, rate_averages_by_movie.loc[i].rating]
            highest_rated.append(movie_rating_temp)
        lowest_rated = rate_averages_by_movie.loc[i].rating

# print(df_movie_rating_temp)
df_highest_rated = pd.DataFrame(highest_rated, columns=['short_title', 'rating'])
print('Highest rated movies')
print(df_highest_rated)


Highest rated movies
                          short_title  rating
0                   Bittersweet Motel     5.0
1                    Follow the Bitch     5.0
2         Gate of Heavenly Peace, The     5.0
3                               Lured     5.0
4                   One Little Indian     5.0
5  Schlafes Bruder (Brother of Sleep)     5.0
6                       Smashing Time     5.0
7                     Song of Freedom     5.0
8                    Ulysses (Ulisse)     5.0


###### What is a good rated movie for date night? (60 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined

In [212]:
# This 'method' produce results based on the number of reviews being at least 75, the average age for the reviewrs has to be 18, 
# as well as the average rating for each movie, and generates a list of the best movies.

num_rev_by_movie_temp = df_merged.short_title.value_counts()
df_movie_index = pd.DataFrame(movie_index, columns=['short_title'])
# print(df_movie_index.loc[986].short_title)
# print(num_rev_by_movie_temp[df_movie_index.loc[986].short_title])
# print(df_movie_index.loc[len(df_movie_index)-1].short_title)
# print(pd.isnull(df_movie_index.loc[986].short_title))
num_rev_by_movie = []
for i in range(len(df_movie_index)-1):
    if pd.isnull(df_movie_index.loc[i].short_title) == False:
        temp = [df_movie_index.loc[i].short_title, num_rev_by_movie_temp[df_movie_index.loc[i].short_title]]
        num_rev_by_movie.append(temp)
        # print('movie_index: ', i, ' ', df_movie_index.loc[i].short_title, " reviews: ", num_rev_by_movie_temp[df_movie_index.loc[i].short_title])

        
df_num_rev_by_movie = pd.DataFrame(num_rev_by_movie, columns=['short_title', 'number_of_reviews'])
# print(df_num_rev_by_movie)

# print(rate_averages_by_movie.loc[1])
# print(df_num_rev_by_movie.loc[df_num_rev_by_movie['short_title'] == rate_averages_by_movie.loc[1].short_title].number_of_reviews)
# if 10 < df_num_rev_by_movie.loc[df_num_rev_by_movie['short_title'] == rate_averages_by_movie.loc[1].short_title].number_of_reviews.item():
#     print('success')


initial_rated = 0
lowest_possible_reviews = 75
best_rated_and_num_reviews = []
for i in range(0, 3840):
     if initial_rated <=  rate_averages_by_movie.loc[i].rating and lowest_possible_reviews<= (df_num_rev_by_movie.loc[df_num_rev_by_movie['short_title'] == rate_averages_by_movie.loc[i].short_title].number_of_reviews.item()) and (rate_averages_by_movie.loc[i].age)>= 18:
        movie_rating_temp2 = [rate_averages_by_movie.loc[i].short_title, rate_averages_by_movie.loc[i].rating]
        best_rated_and_num_reviews.append(movie_rating_temp2)
        initial_rated = rate_averages_by_movie.loc[i].rating
            

# print(best_rabest_rated_and_num_reviewsted)
df_best_rated_and_num_reviews = pd.DataFrame(best_rated_and_num_reviews, columns=['short_title', 'rating'])
print('Best movies for a date night:')
print(df_best_rated_and_num_reviews)


Best movies for a date night:
                                         short_title    rating
0                                        'burbs, The  2.910891
1                             ...And Justice for All  3.713568
2                                       12 Angry Men  4.295455
3            400 Blows, The (Les Quatre cents coups)  4.320856
4           Bicycle Thief, The (Ladri di biciclette)  4.357143
5                      Bridge on the River Kwai, The  4.386994
6                                         Casablanca  4.412822
7                                     Close Shave, A  4.520548
8                                     Godfather, The  4.524966
9  Seven Samurai (The Magnificent Seven) (Shichin...  4.560510
